In [1]:
import boto3
import pandas as pd
import pyarrow.parquet as pq
import io
import os

In [2]:
access_key = os.getenv('AWS_ACCESS_KEY_ID')
secret_key = os.getenv('AWS_SECRET_ACCESS_KEY')

In [3]:
# S3 클라이언트 생성 (환경변수에서 자격증명 자동으로 가져옴)
s3 = boto3.client('s3', region_name='ap-northeast-2')

In [4]:
bucket_name = 'aws-s3-bucket-fastcampus'
prefix = 'dataLake_upbit/'  # Parquet 파일들이 저장된 경로

In [5]:
# S3에서 지정된 경로에 있는 모든 파일 목록 가져오기
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

In [6]:
# 파일 목록에서 Parquet 파일들의 Key 추출
parquet_files = [obj['Key'] for obj in response['Contents'] if obj['Key'].endswith('.parquet')]

In [7]:
# 모든 Parquet 파일을 DataFrame으로 변환하고 하나로 병합
dataframes = []
for i, file_key in enumerate(parquet_files):
    obj = s3.get_object(Bucket=bucket_name, Key=file_key)
    data = obj['Body'].read()
    
    table = pq.read_table(io.BytesIO(data))
    df = table.to_pandas()
    dataframes.append(df)

In [8]:
combined_df = pd.concat(dataframes, ignore_index=True)
print(combined_df.head())

      code trade_date trade_time  trade_timestamp  high_price   low_price  \
0  KRW-CTC   20241007     190853    1728328133932       588.5       557.1   
1  KRW-CTC   20241007     190853    1728328133949       588.5       557.1   
2  KRW-BTC   20241007     190854    1728328134487  86301000.0  84150000.0   
3  KRW-GRS   20241007     190854    1728328134699       482.0       465.7   
4  KRW-GRS   20241007     190854    1728328134699       482.0       465.7   

   trade_price change  change_price  change_rate      timestamp  
0        567.8   RISE           7.7     0.013748  1728328133965  
1        567.8   RISE           7.7     0.013748  1728328133986  
2   85099000.0   RISE      566000.0     0.006696  1728328134515  
3        468.6   FALL          11.9     0.024766  1728328134720  
4        468.4   FALL          12.1     0.025182  1728328134720  


In [9]:
combined_df.to_csv('upbit_combined.csv', index=False)